# И теперь задеплоим что нибудь!

Открываем гугл клауд, слева вверху настройки, выбираем VPC Network и Firewall Rules

<img src="external_ip.png" alt="hop hop loshok" height="1000" width="1000">

# Потом настраиваем firewall

нажимаем create firewall rule, задаем уникальное имя, direction ставим ingress (то есть входящий трафик)    
action on match ставим allow, target tags ставим dash-app (это штука с помощью которой к нашей вирт машине будут применяться правила)  
Source IP ranges ставим все нули 0.0.0.0/0 (откуда может залетать трафик, все нули, значит с любого)  
  


# Создаем внешний ай пи

Открываем гугл клауд, слева вверху настройки, выбираем VPC Network и external ip addresses  
reserve a static address  
указываем название,   
network service Tier ставим standard  
ip version 4  
и резервируем  

# ну а дальше вирутальная машинка

снова идем в настройки гугл клауд и создаем instance  
выбираем в меню Compute Engine, VM instance    
create instance  
указываем имя и настройки   
Firewall ставим все галочки разрешить  
далее нажимаем после блока Firewall
Networking  
в Network tags указываем как в firewall rules указали в target tags, то есть ставим dash-app  
Далее ниже  
network interfaces  
нажимаем show alias ip ranges и выбираем наш созданный external ip (там будет в списке)  
далее done и create

теперь заходим в ssh вирт машины и выполняем следущие команды

In [ ]:
# теперь обновляем систему и устанавливаем нужные пакеты
sudo apt update
sudo apt install python3-pip
# Для работы с виртуальным окружением в Linux нужно установить пакет venv с помощью следующей команды:
sudo apt install python3-venv

mkdir dash_app
cd dash_app

# создаем виртуальную питоновскую среду
# Создать виртуальное окружение можно командой python3 -m venv my_venv.
# My_venv – название виртуального окружения.
# Приведенная выше команда создает каталог с именем «my_venv» (а также родительские каталоги, которые ещё не существуют), 
# содержащий менеджер пакетов pip, интерпретатор, скрипты и библиотеки.
# эта папка и будет виртуальня среда разработки
python3 -m venv dash_venv
# # С помощью команды ls -la можно увидеть файлы в текущей директории.
# Файл pyvenv.cfg содержит ключ, который будет указывать на ту версию Python, для которой запущена эта команда. 

# Каталог bin содержит копию/символическую ссылку двоичных файлов Python.

# Каталог include включает в себя С-заголовки, компилирующие пакеты Python.

# В каталог share входит python wheels. Python wheels — это формат готовых пакетов для Python, который помогает ускорить разработку 
# программного обеспечения за счет сокращения количества операций компиляции. 

# Каталог lib в виртуальной среде имеет такую же структуру, как каталог lib в глобальном окружении. 
# И содержит папку site-packages, в которую устанавливаются библиотеки.

# Чтобы начать использовать эту виртуальную среду, вам необходимо активировать ее, запустив скрипт под названием activate:
source dash_venv/bin/activate  
# После активации строка консоли будет иметь префикс с именем среды.
# После активации все библиотеки будут устанавливаться в данное виртуальное окружение.

# Wheel (.whl) — это бинарный формат дистрибутива, который позволяет устанавливать пакеты Python без компиляции исходного кода.
pip install wheel
pip install dash==1.0.2 dash-daq==0.1.0 pyorbital plotly pandas flask gunicorn pyopenssl

# если нужно деативировать среду, то пишем в консоли
deactivate

# чтобы войти в среду входим в папку, где лежит папка среды и пишем как и при создании
source dash_venv/bin/activate  

теперь помещаем код ниже в файл с расширение .py    
и загружаем на вирт машину гугла, для этого в   
ssh нажимаем шестеренку и upload filem  

In [ ]:
# Flask — это упрощенная платформа Python для веб-приложений, которая обеспечивает основные возможности маршрутизации URL-адресов и визуализации страниц. 
from flask import Flask
app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World!"

if __name__ == "__main__":
    app.run(
        ssl_context='adhoc',
        host='0.0.0.0',
        port='8080'
        )

In [ ]:
# далее в консоли набираем  
python имя_файла_с_предыдущего_шага.py

In [ ]:
# появится адерс с нулями и главное порт  
# далее берем наш внеший ай пи вирт машины (в настройках вирт машины в гугл клауд external ip)  
# и добавляем к нему порт через двоеточие, типа такого
x.x.x.x:port
# и вбиваем в браузер
# будет сообщение. что соединение not private  
# по хорошему нужно настроить ssl сертификат
# но для обучения нажимаем просто  что согласны с опасностью

можно создать сертификат

переходим в папку dash_app (в этой папке наша установленная среда в виде папки dash_venv)  
и выполняем следующую команду

In [ ]:
openssl req -x509 -newkey rsa:4096 -nodes -out cert.pem -keyout key.pem -days 365

указываем 
RU
Moscow
и остальное любое

теперь чтобы наш дашборд работал,  
загружаем через ssh через меню настроек и upload file  
нужный файл питона с дашбордом.  
загрузится в корневую папку витр машины  


In [ ]:
# далее перемещаем его в папку нашей витр среды
# выходим в корневую папку и копируем текущий адрес
pwd 
# копируем что вывелось
cp -i app.py скорированный_адрес скорированный_адрес/dash_app

In [ ]:
# далее переходим в папку dash_app и запускаем
python app.py